In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing surprise module
!pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096350 sha256=02aa892f06a7d0e785a1d4df5c99cb777a98ffadb6e3a94ab79db64bfa91cdad
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import numpy as np
import pandas as pd

# Importing libraries for model building & evaluation
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from surprise import NMF, SVD
from sklearn.metrics import precision_recall_fscore_support as prf
from surprise import accuracy

In [4]:
#reading the dataset
df = pd.read_csv('/content/drive/MyDrive/testcombinedData.csv')
df.head()

,userId,rating,tmdbId,title,genres
0,1,2.5,9909.0,Dangerous Minds (1995),Drama
1,7,3.0,9909.0,Dangerous Minds (1995),Drama
2,31,4.0,9909.0,Dangerous Minds (1995),Drama
3,32,4.0,9909.0,Dangerous Minds (1995),Drama
4,36,3.0,9909.0,Dangerous Minds (1995),Drama


In [6]:
# Load the dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'tmdbId', 'rating']], reader)

# Train the SVD model on the entire dataset
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [11]:
def user_based_recommendations(userId, num_recom):
  # unrated items from user
  unrated_items = [item_id for item_id in trainset.all_items() if item_id not in trainset.ur[trainset.to_inner_uid(userId)]]

  # Predict ratings
  predictions = [(item_id, algo.predict(userId, item_id).est) for item_id in unrated_items]

  # Sort the predicted ratings in descending order
  predictions.sort(key=lambda x: x[1], reverse=True)

  # get the top-N recommendations
  top_n = predictions[:num_recom]

  #as these recommendations are of form movie id
  #fetch movietitles using this movie ids using dataset
  titles = []
  for movie_id, bias in top_n:
          title = list(df.title[df['tmdbId']==movie_id])
          titles.append(title[0])
  return titles



In [13]:
user_based_recommendations(576, 10)

['Shawshank Redemption, The (1994)',
 'Godfather: Part II, The (1974)',
 'Godfather, The (1972)',
 'Cool Hand Luke (1967)',
 'American Beauty (1999)',
 'On the Waterfront (1954)',
 'Roger & Me (1989)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Treasure of the Sierra Madre, The (1948)',
 'Young Frankenstein (1974)']

In [14]:
user_based_recommendations(123, 10)

['Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'City of God (Cidade de Deus) (2002)',
 'Raging Bull (1980)',
 'Usual Suspects, The (1995)',
 "Singin' in the Rain (1952)",
 'All About Eve (1950)',
 'Life Is Beautiful (La Vita è bella) (1997)',
 'Third Man, The (1949)',
 'Fargo (1996)']